<a href="https://colab.research.google.com/github/Kuchurusiddhartha/CodeSid/blob/main/FakeNewsDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install pandas scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


df = pd.read_csv("fake_news_dataset[1].csv")


df = df.dropna(subset=["text", "label"])


print("Unique labels before mapping:", df["label"].unique())


def clean_label(x):
    if str(x).lower() in ["fake", "1"]:
        return 1
    elif str(x).lower() in ["true", "real", "0"]:
        return 0
    else:
        return None

df["label"] = df["label"].apply(clean_label)


df = df.dropna(subset=["label"])
df["label"] = df["label"].astype(int)


X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)


tfidf = TfidfVectorizer(stop_words="english", max_df=0.7)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(X_train_tfidf, y_train)


y_pred = pac.predict(X_test_tfidf)
score = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {round(score*100,2)}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred, labels=[0, 1]))


sample_news = [
    "The government announced a new plan for economic recovery.",
    "Breaking! Aliens just landed in New York according to anonymous sources."
]

sample_vec = tfidf.transform(sample_news)
sample_pred = pac.predict(sample_vec)

for i, news in enumerate(sample_news):
    print("\nNews:", news)
    print("Prediction:", "Fake" if sample_pred[i] == 1 else "True")


Unique labels before mapping: ['Fake' 'Real']

Accuracy: 51.3%
Confusion Matrix:
[[  0 487]
 [  0 513]]

News: The government announced a new plan for economic recovery.
Prediction: Fake

News: Breaking! Aliens just landed in New York according to anonymous sources.
Prediction: Fake
